In [1]:
!pip install natsort

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import glob
import cv2
from natsort import natsorted

%matplotlib inline

In [3]:
!ls /kaggle/input/

planet-understanding-the-amazon-from-space  planets-dataset


In [4]:
!ls /kaggle/input/planets-dataset/planet/planet/

sample_submission.csv  test-jpg  train-jpg  train_classes.csv


In [5]:
train_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')
train_df.head(10)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


In [6]:
train_df.tail(10)

,image_name,tags
40469,train_40469,haze primary water
40470,train_40470,clear habitation primary road
40471,train_40471,clear primary
40472,train_40472,agriculture clear primary
40473,train_40473,agriculture clear habitation primary road
40474,train_40474,clear primary
40475,train_40475,cloudy
40476,train_40476,agriculture clear primary
40477,train_40477,agriculture clear primary road
40478,train_40478,agriculture cultivation partly_cloudy primary


In [7]:
sample_submission_df = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')
sample_submission_df.head(10)

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water
5,test_5,primary clear agriculture road water
6,test_6,primary clear agriculture road water
7,test_7,primary clear agriculture road water
8,test_8,primary clear agriculture road water
9,test_9,primary clear agriculture road water


In [8]:
sample_submission_df.tail(10)

,image_name,tags
61181,file_9990,primary clear agriculture road water
61182,file_9991,primary clear agriculture road water
61183,file_9992,primary clear agriculture road water
61184,file_9993,primary clear agriculture road water
61185,file_9994,primary clear agriculture road water
61186,file_9995,primary clear agriculture road water
61187,file_9996,primary clear agriculture road water
61188,file_9997,primary clear agriculture road water
61189,file_9998,primary clear agriculture road water
61190,file_9999,primary clear agriculture road water


In [9]:
!ls /kaggle/input/planet-understanding-the-amazon-from-space/

Kaggle-planet-test-tif.torrent	 sample_submission_v2.csv  train_v2.csv
Kaggle-planet-train-tif.torrent  test_v2_file_mapping.csv


In [10]:
label_list = []
for tag_str in train_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [11]:
for label in label_list:
    train_df[label] = train_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)

train_df.sample(10)

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
1352,train_1352,clear cultivation primary water,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
2365,train_2365,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27353,train_27353,partly_cloudy primary,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2714,train_2714,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
37262,train_37262,agriculture clear primary road,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
21917,train_21917,clear cultivation primary,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
27921,train_27921,cloudy,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1482,train_1482,agriculture clear cultivation primary,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
23567,train_23567,partly_cloudy primary,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24010,train_24010,partly_cloudy primary,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [12]:
target_list = train_df.drop(columns=['image_name','tags']).columns.tolist()
target_list

['haze',
 'primary',
 'agriculture',
 'clear',
 'water',
 'habitation',
 'road',
 'cultivation',
 'slash_burn',
 'cloudy',
 'partly_cloudy',
 'conventional_mine',
 'bare_ground',
 'artisinal_mine',
 'blooming',
 'selective_logging',
 'blow_down']

In [13]:
train_df['image_file'] = train_df['image_name']+'.jpg'
train_df.head(10)

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,image_file
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,train_0.jpg
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,train_1.jpg
2,train_2,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,train_2.jpg
3,train_3,clear primary,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,train_3.jpg
4,train_4,agriculture clear habitation primary road,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,train_4.jpg
5,train_5,haze primary water,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,train_5.jpg
6,train_6,agriculture clear cultivation primary water,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,train_6.jpg
7,train_7,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,train_7.jpg
8,train_8,agriculture clear cultivation primary,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,train_8.jpg
9,train_9,agriculture clear cultivation primary road,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,train_9.jpg


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40479 entries, 0 to 40478
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_name         40479 non-null  object
 1   tags               40479 non-null  object
 2   haze               40479 non-null  int64 
 3   primary            40479 non-null  int64 
 4   agriculture        40479 non-null  int64 
 5   clear              40479 non-null  int64 
 6   water              40479 non-null  int64 
 7   habitation         40479 non-null  int64 
 8   road               40479 non-null  int64 
 9   cultivation        40479 non-null  int64 
 10  slash_burn         40479 non-null  int64 
 11  cloudy             40479 non-null  int64 
 12  partly_cloudy      40479 non-null  int64 
 13  conventional_mine  40479 non-null  int64 
 14  bare_ground        40479 non-null  int64 
 15  artisinal_mine     40479 non-null  int64 
 16  blooming           40479 non-null  int64

In [15]:
training_df = train_df[:int(len(train_df)*0.7)]
validation_df = train_df[int(len(train_df)*0.7):int(len(train_df)*0.9)]
synth_test_df = train_df[int(len(train_df)*0.9):]

In [16]:
len(training_df), len(validation_df), len(synth_test_df)

(28335, 8096, 4048)

In [17]:
len(train_df), (len(training_df) + len(validation_df) + len(synth_test_df))

(40479, 40479)

In [18]:
batch_size = 64

image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     rotation_range=15, 
                                     width_shift_range=.15, 
                                     height_shift_range=.15, 
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     zoom_range=0.2
                                    )


train_data_gen = image_gen_train.flow_from_dataframe(training_df, 
                                                     directory='/kaggle/input/planets-dataset/planet/planet/train-jpg', 
                                                     x_col='image_file', 
                                                     y_col=target_list, 
                                                     class_mode='raw', 
                                                     target_size=(256,256),
                                                     batch_size=batch_size)

Found 28335 validated image filenames.


In [19]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_dataframe(validation_df, 
                                                   directory='/kaggle/input/planets-dataset/planet/planet/train-jpg', 
                                                   x_col='image_file', 
                                                   y_col=target_list,
                                                   class_mode='raw', 
                                                   shuffle=False,
                                                   target_size=(256,256),
                                                   batch_size=batch_size)

Found 8096 validated image filenames.


In [20]:
image_gen_synth_test = ImageDataGenerator(rescale=1./255)

synth_test_data_gen = image_gen_synth_test.flow_from_dataframe(synth_test_df, 
                                                   directory='/kaggle/input/planets-dataset/planet/planet/train-jpg', 
                                                   x_col='image_file', 
                                                   class_mode=None, 
                                                   shuffle=False,
                                                   target_size=(256,256),
                                                   batch_size=1)

Found 4048 validated image filenames.


In [21]:
transfer_learning_layer = tf.keras.applications.InceptionResNetV2(include_top=False, 
                                                      weights='imagenet', 
                                                      input_shape=(256,256,3)
                                                     )

219062272/219055592 [==============================] - 1s 0us/step


In [22]:
transfer_learning_layer.trainable = False

data_augmentation_layer = Sequential(
[
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
    tf.keras.layers.experimental.preprocessing.RandomFlip(),
    tf.keras.layers.experimental.preprocessing.RandomWidth(.15),
    tf.keras.layers.experimental.preprocessing.RandomHeight(.15),
    tf.keras.layers.experimental.preprocessing.RandomZoom(.35),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

NASNet_layer = tf.keras.applications.NASNetLarge(include_top=False, 
                                                 weights='imagenet', 
                                                 input_shape=(331,331,3)
                                                )

NASNet_layer.trainable = False

URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
mobile_net_layer = hub.KerasLayer(URL,
                                   input_shape=(256, 256,3))

In [23]:
model = Sequential([
    transfer_learning_layer,
    Dropout(0.2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(17, activation='sigmoid')
])

In [24]:
model.compile(optimizer='Adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
              metrics=['accuracy']
             )

In [25]:
epochs = 3
history = model.fit(train_data_gen, 
                    epochs=epochs, 
                    validation_data=val_data_gen)

Epoch 1/3
443/443 [==============================] - 572s 1s/step - loss: 0.3228 - accuracy: 0.7336 - val_loss: 0.1404 - val_accuracy: 0.8373
Epoch 2/3
443/443 [==============================] - 522s 1s/step - loss: 0.1491 - accuracy: 0.8275 - val_loss: 0.1280 - val_accuracy: 0.8103
Epoch 3/3
443/443 [==============================] - 521s 1s/step - loss: 0.1430 - accuracy: 0.8244 - val_loss: 0.1262 - val_accuracy: 0.7908


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
dropout (Dropout)            (None, 6, 6, 1536)        0         
_________________________________________________________________
flatten (Flatten)            (None, 55296)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              56624128  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                17425     
Total params: 110,978,289
Trainable params: 56,641,553
Non-trainable params: 54,336,736
__________________________________

In [27]:
synth_test_data_gen.reset()
synth_test_pred = model.predict_generator(synth_test_data_gen, verbose=1)

4048/4048 [==============================] - 135s 33ms/step


In [28]:
synth_test_pred_bool = (synth_test_pred >0.5)

In [29]:
synth_test_predictions = synth_test_pred_bool.astype(int)
columns=target_list
#columns should be the same order of y_col
synth_test_results = pd.DataFrame(synth_test_predictions, columns=columns)
synth_test_results["image_name"] = synth_test_data_gen.filenames
ordered_cols = ["image_name"] + columns
synth_test_results = synth_test_results[ordered_cols]#To get the same column order
synth_test_results

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_36431.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_36432.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,train_36433.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,train_36434.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
4,train_36435.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,train_40474.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4044,train_40475.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4045,train_40476.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4046,train_40477.jpg,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [30]:
synth_test_results['tags'] = ''

for col_name in synth_test_results[target_list].columns:
    synth_test_results.loc[synth_test_results[col_name]==1,'tags'] = synth_test_results['tags']+' '+col_name
synth_test_results

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,tags
0,train_36431.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
1,train_36432.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
2,train_36433.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,primary agriculture road partly_cloudy
3,train_36434.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,primary clear water
4,train_36435.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,primary clear water
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4043,train_40474.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
4044,train_40475.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,primary partly_cloudy
4045,train_40476.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary agriculture clear
4046,train_40477.jpg,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,primary agriculture clear road


In [31]:
synth_test_results = synth_test_results.drop(columns=target_list)
synth_test_results

,image_name,tags
0,train_36431.jpg,primary clear
1,train_36432.jpg,primary clear
2,train_36433.jpg,primary agriculture road partly_cloudy
3,train_36434.jpg,primary clear water
4,train_36435.jpg,primary clear water
...,...,...
4043,train_40474.jpg,primary clear
4044,train_40475.jpg,primary partly_cloudy
4045,train_40476.jpg,primary agriculture clear
4046,train_40477.jpg,primary agriculture clear road


In [32]:
from sklearn.metrics import accuracy_score

In [33]:
synth_test_labels = synth_test_df.drop(columns=['image_name', 'tags', 'image_file'])
synth_test_labels

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
36431,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
36432,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
36433,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
36434,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0
36435,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
40475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
40476,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
40477,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [34]:
synth_test_labels['tags'] = ''

for col_name in synth_test_labels.columns:
    synth_test_labels.loc[synth_test_labels[col_name]==1,'tags'] = synth_test_labels['tags']+' '+col_name
synth_test_labels

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,tags
36431,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
36432,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
36433,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,primary agriculture road partly_cloudy
36434,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,primary clear water habitation
36435,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,primary clear water road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40474,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
40475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,cloudy
40476,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary agriculture clear
40477,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,primary agriculture clear road


In [35]:
synth_test_labels = synth_test_labels.drop(columns=target_list)
synth_test_labels

,tags
36431,primary clear
36432,primary clear
36433,primary agriculture road partly_cloudy
36434,primary clear water habitation
36435,primary clear water road
...,...
40474,primary clear
40475,cloudy
40476,primary agriculture clear
40477,primary agriculture clear road


In [36]:
accuracy_score(synth_test_labels, synth_test_results['tags'])

0.5592885375494071

In [37]:
test_images_list = []

for filename in glob.glob('../input/planets-dataset/planet/planet/test-jpg'+'/*.jpg'):
    test_images_list.append(os.path.basename(filename))

test_df = pd.DataFrame(test_images_list, columns=['filename'])
test_df

,filename
0,test_32347.jpg
1,test_29340.jpg
2,test_33303.jpg
3,test_4963.jpg
4,test_1886.jpg
...,...
40664,test_13668.jpg
40665,test_23623.jpg
40666,test_6932.jpg
40667,test_26723.jpg


In [38]:
additional_test_images = []

for filename in glob.glob('../input/planets-dataset/test-jpg-additional/test-jpg-additional'+'/*.jpg'):
    additional_test_images.append(os.path.basename(filename))

add_test_df = pd.DataFrame(additional_test_images, columns=['filename'])
add_test_df 

,filename
0,file_9603.jpg
1,file_14712.jpg
2,file_9163.jpg
3,file_12141.jpg
4,file_2428.jpg
...,...
20517,file_2609.jpg
20518,file_12444.jpg
20519,file_7403.jpg
20520,file_19350.jpg


In [39]:
test_df_full = test_df.append(add_test_df, ignore_index=True)
test_df_full

,filename
0,test_32347.jpg
1,test_29340.jpg
2,test_33303.jpg
3,test_4963.jpg
4,test_1886.jpg
...,...
61186,file_2609.jpg
61187,file_12444.jpg
61188,file_7403.jpg
61189,file_19350.jpg


In [40]:
image_gen_test = ImageDataGenerator(rescale=1./255)

test_data_gen_1 = image_gen_test.flow_from_dataframe(test_df, 
                                                   directory='../input/planets-dataset/planet/planet/test-jpg', 
                                                   x_col='filename', 
                                                   class_mode=None, 
                                                   shuffle=False,
                                                   target_size=(256,256),
                                                   batch_size=1)

test_data_gen_2 = image_gen_test.flow_from_dataframe(add_test_df, 
                                                   directory='../input/planets-dataset/test-jpg-additional/test-jpg-additional', 
                                                   x_col='filename', 
                                                   class_mode=None, 
                                                   shuffle=False,
                                                   target_size=(256,256),
                                                   batch_size=1)

Found 40669 validated image filenames.
Found 20522 validated image filenames.


In [41]:
test_data_gen_1.reset()
pred_1 = model.predict_generator(test_data_gen_1, verbose=1)

40669/40669 [==============================] - 1367s 34ms/step


In [42]:
pred_bool = (pred_1 >0.5)

In [43]:
predictions = pred_bool.astype(int)
columns = target_list
results = pd.DataFrame(predictions, columns=columns)
results["image_name"] = test_data_gen_1.filenames
ordered_cols = ["image_name"] + columns
results = results[ordered_cols]
results

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,test_32347.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,test_29340.jpg,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,test_33303.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,test_4963.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,test_1886.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40664,test_13668.jpg,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
40665,test_23623.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
40666,test_6932.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
40667,test_26723.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
results.image_name = results.image_name.astype('category')

In [45]:
results.image_name.cat.reorder_categories(natsorted(results.image_name), inplace=True, ordered=True)

In [46]:
results = results.sort_values(by='image_name', ignore_index=True)
results

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,test_0.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,test_1.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,test_2.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,test_3.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,test_4.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40664,test_40664.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
40665,test_40665.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
40666,test_40666.jpg,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
40667,test_40667.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [47]:
test_data_gen_2.reset()
pred_2 = model.predict_generator(test_data_gen_2, verbose=1)

20522/20522 [==============================] - 683s 33ms/step


In [48]:
pred_bool_2 = (pred_2 >0.5)
predictions_2 = pred_bool_2.astype(int)
results_2 = pd.DataFrame(predictions_2, columns=columns)
results_2["image_name"] = test_data_gen_2.filenames
results_2 = results_2[ordered_cols]
results_2

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,file_9603.jpg,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,file_14712.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,file_9163.jpg,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,file_12141.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,file_2428.jpg,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20517,file_2609.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20518,file_12444.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20519,file_7403.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20520,file_19350.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
results_2.image_name = results_2.image_name.astype('category')

In [50]:
results_2.image_name.cat.reorder_categories(natsorted(results_2.image_name), inplace=True, ordered=True)

In [51]:
results_2 = results_2.sort_values(by='image_name', ignore_index=True)
results_2

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,file_0.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,file_1.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,file_2.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,file_3.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,file_4.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20517,file_20517.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
20518,file_20518.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
20519,file_20519.jpg,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
20520,file_20520.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
submission_df = pd.concat([results, results_2], ignore_index=True, copy=False)
submission_df

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,test_0.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,test_1.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,test_2.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,test_3.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,test_4.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,file_20517.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
61187,file_20518.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
61188,file_20519.jpg,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
61189,file_20520.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
submission_df['tags'] = ''

for col_name in submission_df[target_list].columns:
    submission_df.loc[submission_df[col_name]==1,'tags'] = submission_df['tags']+' '+col_name
submission_df

,image_name,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down,tags
0,test_0.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
1,test_1.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
2,test_2.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,primary partly_cloudy
3,test_3.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
4,test_4.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,primary partly_cloudy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,file_20517.jpg,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary clear
61187,file_20518.jpg,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,primary partly_cloudy
61188,file_20519.jpg,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,primary clear road
61189,file_20520.jpg,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,primary agriculture clear


In [54]:
submission_df = submission_df.drop(columns=target_list)
submission_df

,image_name,tags
0,test_0.jpg,primary clear
1,test_1.jpg,primary clear
2,test_2.jpg,primary partly_cloudy
3,test_3.jpg,primary clear
4,test_4.jpg,primary partly_cloudy
...,...,...
61186,file_20517.jpg,primary clear
61187,file_20518.jpg,primary partly_cloudy
61188,file_20519.jpg,primary clear road
61189,file_20520.jpg,primary agriculture clear


In [55]:
submission_df['image_name'] = submission_df['image_name'].apply(lambda x: x.strip('.jpg'))
submission_df

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary clear
4,test_4,primary partly_cloudy
...,...,...
61186,file_20517,primary clear
61187,file_20518,primary partly_cloudy
61188,file_20519,primary clear road
61189,file_20520,primary agriculture clear


In [56]:
!rm submission.csv

rm: cannot remove 'submission.csv': No such file or directory


In [57]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)